In [58]:
# Author:
# Van Thanh Nguyen

import pinocchio as pin
import numpy as np
from SysIDUtils import helpers as sysid_helpers
from sys import argv
from pathlib import Path

# Reload the module to pick up any changes
import importlib
importlib.reload(sysid_helpers)

<module 'SysIDUtils.helpers' from '/home/vanthanhnguyen/Documents/PROGRAMS/Bipedal_Robot_Sytem_Identification/SysIDUtils/helpers.py'>

Define Path to urdf model and

Create pinocchio model and data

In [59]:
import os

# Get the directory where the notebook is located
notebook_dir = os.getcwd()
print(f"Current working directory: {notebook_dir}")

# Build path to the robot model
package_dirs = Path(notebook_dir) / "robot_models" / "left_leg"
urdf_filename = package_dirs / "left_leg.urdf"

print(f"URDF path: {urdf_filename}")
print(f"URDF exists: {urdf_filename.exists()}")

# 1. create model
model = pin.buildModelFromUrdf(str(urdf_filename))
print("model name: " + model.name)
# create data
data = model.createData()

# infere some basic information of the model
print("number of joints (njoints): " + str(model.njoints))
print("number of dofs (nq): " + str(model.nq))
print("number of bodies: " + str(len(model.inertias)-1))  #exclude the universe body
print("number of actuated joints (nv): " + str(model.nv))

# Print all joint names to understand the structure
print("\nJoint names:")
for i, name in enumerate(model.names):
    print(f"  Joint {i}: {name}")

Current working directory: /home/vanthanhnguyen/Documents/PROGRAMS/Bipedal_Robot_Sytem_Identification
URDF path: /home/vanthanhnguyen/Documents/PROGRAMS/Bipedal_Robot_Sytem_Identification/robot_models/left_leg/left_leg.urdf
URDF exists: True
model name: left_leg
number of joints (njoints): 7
number of dofs (nq): 6
number of bodies: 6
number of actuated joints (nv): 6

Joint names:
  Joint 0: universe
  Joint 1: left_hip_pitch
  Joint 2: left_hip_roll
  Joint 3: left_hip_yaw
  Joint 4: left_knee
  Joint 5: left_ankle_roll
  Joint 6: left_ankle_pitch


Get standard/full parameters vector of the model 

In [60]:
# generate a list containing the full set of standard parameters
standard_params = sysid_helpers.get_standard_parameters(model) # return a library of standard parameters including symbols and values
standard_param_symbols_list = sysid_helpers.get_list_standard_param_symbols(model) # return a list of standard parameter symbols
print("\n total of Standard parameters: " + str(len(standard_param_symbols_list)))
print("\nStandard parameter symbols list: ")
print(standard_param_symbols_list)


 total of Standard parameters: 60

Standard parameter symbols list: 
['m1', 'mx1', 'my1', 'mz1', 'Ixx1', 'Ixy1', 'Ixz1', 'Iyy1', 'Iyz1', 'Izz1', 'm2', 'mx2', 'my2', 'mz2', 'Ixx2', 'Ixy2', 'Ixz2', 'Iyy2', 'Iyz2', 'Izz2', 'm3', 'mx3', 'my3', 'mz3', 'Ixx3', 'Ixy3', 'Ixz3', 'Iyy3', 'Iyz3', 'Izz3', 'm4', 'mx4', 'my4', 'mz4', 'Ixx4', 'Ixy4', 'Ixz4', 'Iyy4', 'Iyz4', 'Izz4', 'm5', 'mx5', 'my5', 'mz5', 'Ixx5', 'Ixy5', 'Ixz5', 'Iyy5', 'Iyz5', 'Izz5', 'm6', 'mx6', 'my6', 'mz6', 'Ixx6', 'Ixy6', 'Ixz6', 'Iyy6', 'Iyz6', 'Izz6']


# Calculate Base parameter vector using numerical method
First, generate a random values for position, velocity and acceleration to construct a regressor matrix

In [61]:
n_samples = 30 # sample size to generate random configurations

# get vectors of joint limits, velocity limits
joint_lower_limits = []
joint_upper_limits = []
joint_velocity_limits = []
joint_lower_limits = model.lowerPositionLimit.copy()
joint_upper_limits = model.upperPositionLimit.copy()
joint_velocity_limits = model.velocityLimit.copy()
joint_lower_limits = np.array(joint_lower_limits)
joint_upper_limits = np.array(joint_upper_limits)
joint_velocity_limits = np.array(joint_velocity_limits)
# default limit qdd to 6*pi rad/s2
joint_acceleration_limits = 6 * np.pi * np.ones(model.nv)

# generate 30 samples of q_rand, dq_rand, ddq_rand
q_rand = np.array([pin.randomConfiguration(model) for _ in range(n_samples)])
dq_rand = np.random.uniform(low=-joint_velocity_limits, high=joint_velocity_limits, size=(n_samples, model.nv))
ddq_rand = np.random.uniform(low=-joint_acceleration_limits, high=joint_acceleration_limits, size=(n_samples, model.nv))

Now we construct a standard regressor matrix

In [62]:
W_standard = sysid_helpers.calculate_standard_regressor(model, data, q_rand, dq_rand, ddq_rand)
print("\nStandard regressor matrix W_standard shape: ", W_standard.shape)

# Infere the rank of the standard regressor matrix
print("\nRank of the standard regressor matrix W_standard: ")
print(np.linalg.matrix_rank(W_standard)) # just for initial checking


Standard regressor matrix W_standard shape:  (180, 60)

Rank of the standard regressor matrix W_standard: 
38


In [63]:
# Compute numerical rank using QR decomposition with column pivoting
# it should be same as np.linalg.matrix_rank
regressor_num_rank = sysid_helpers.compute_numerical_rank(W_standard)
print("\nNumerical rank of the standard regressor matrix W_standard: ")
print(regressor_num_rank)


Numerical rank of the standard regressor matrix W_standard: 
38


compute base parameter vector

In [64]:
X_base, beta, independent_idx, dependent_idx = sysid_helpers.calculate_base_parameters_symbols(W_standard,
                                                                                               standard_param_symbols_list,
                                                                                                 TOL_QR=1e-6)
# X_base: base parameter vector
# beta: coefficient matrix for dependent parameters
# independent_idx: indices of independent parameters in param_st
# dependent_idx: indices of dependent parameters in param_st

# print("Base parameter vector length: ", len(X_base) ) # len(X_base) should be equal to regressor_num_rank
# for param in X_base:
#     print(param)

Relationship in regressor matrix found
Max residual in base parameter calculation:  7.793321543658749e-12


In [ ]:
# Now we verify the above computation by performing inverse dynamics calculations
# Generate sine trajectory for each joint
